In [1]:
import multiprocessing

In [2]:
def test(name):
    print(f'here {name}')

In [3]:
processes = []
for i in range(5):
    t = multiprocessing.Process(target=test, args=(i,))
    processes.append(t)
    t.start()
    
for process in processes:
    process.join()

In [4]:
import os

In [5]:
def test(name):
    print(f'here {name}, process: {os.getpid()}, parent process: {os.getppid()}')

In [6]:
processes = []
for i in range(5):
    t = multiprocessing.Process(target=test, args=(i,))
    processes.append(t)
    t.start()
    
for process in processes:
    process.join()

In [7]:
result = [] 
  
def square_list(mylist): 
    global result
    for num in mylist: 
        result.append(num * num)
    print(f"Result(in process p1): {result}")

In [8]:
temp = [1,2,3,4]

p1 = multiprocessing.Process(target=square_list, args=(temp, ))
p1.start()
p1.join()


In [9]:
print(result)

[]


When doing concurrent programming, it is usually best to avoid using shared state as much as possible. This is particularly true when using multiple processes.

However, if you really do need to use some shared data, then multiprocessing provides a couple of ways to do so.

In [10]:
def f(n, a):
    n.value = 3.1415927
    for i in range(len(a)):
        a[i] = -a[i]

In [11]:
num = multiprocessing.Value('d', 0.0)
arr = multiprocessing.Array('i', range(10))

The `d` and `i` arguments, used when creating num and arr variables, are typecodes of the kind used by the array module: `d` indicates a double precision float, and `i` indicates a signed integer. These shared objects will be process- and thread-safe.

In [12]:
print(num.value)
print(arr)

0.0
<SynchronizedArray wrapper for <multiprocessing.sharedctypes.c_long_Array_10 object at 0x000001D2B60B8CD0>>


In [13]:
p = multiprocessing.Process(target=f, args=(num, arr))
p.start()
p.join()

print(num.value)
print(arr[:])

0.0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [14]:
def square_list(mylist, res): 
    global result
    for ind, num in enumerate(mylist): 
        res[ind]= num * num
    print(f"Result(in process p1): {result[:]}")

In [15]:
temp = [1,2,3,4]
result = multiprocessing.Array('i', 4)

p1 = multiprocessing.Process(target=square_list, args=(temp, result))
p1.start()
p1.join()


In [16]:
print(result[:])

[0, 0, 0, 0]


The effective use of multiple processes usually requires some communication between them, so that work can be divided and results can be aggregated. 
Multiprocessing supports two types of communication channels between processes:

- Queue
- Pipe

In [17]:
def square_list(mylist, q): 
    for num in mylist: 
        q.put(num * num) 

def read_from_queue(q):
    while not q.empty():
        print(q.get())
        
        

In [18]:
temp = [1, 2, 3, 4]
q = multiprocessing.Queue()
p1 = multiprocessing.Process(target=square_list, args=(temp, q))
p2 = multiprocessing.Process(target=read_from_queue, args=(q,))

p1.start()
p2.start()
p1.join()
p2.join()

In [19]:
def sender(conn, msgs): 
    for msg in msgs: 
        conn.send(msg) 
        print(f"Sent the message: {msg} from process {os.getpid()}") 
    conn.close() 


def receiver(conn): 
    while True: 
        msg = conn.recv() 
        if msg == "END": 
            break
        print(f"Received the message: {msg} in process {os.getpid()}")

msgs = ["hello", "hey", "hru?", "END"] 
  
# creating a pipe 
parent_conn, child_conn = multiprocessing.Pipe() 
  
# creating new processes 
p1 = multiprocessing.Process(target=sender, args=(parent_conn,msgs)) 
p2 = multiprocessing.Process(target=receiver, args=(child_conn,)) 

p1.start() 
p2.start() 
p1.join() 
p2.join()

The `Pool` class can be used to manage a fixed number of workers for simple cases where the work to be done can be broken up and distributed between the workers independently. The return values from the jobs are collected and returned as a list. The pool arguments include the number of processes, and a function to run when starting the task process (invoked once per child).

In [20]:
def f(x):
    return x * x

In [21]:
import time

In [22]:
with multiprocessing.Pool(processes=4) as pool:
    result = pool.apply_async(f, (10,)) # evaluate "f(10)" asynchronously in a single process
    print(result.get(timeout=1))        # prints "100" unless your computer is *very* slow

    print(pool.map(f, range(10)))       # prints "[0, 1, 4,..., 81]"

    it = pool.imap(f, range(10))
    print(next(it))                     # prints "0"
    print(next(it))                     # prints "1"
    print(it.next(timeout=1))           # prints "4" unless your computer is *very* slow

    result = pool.apply_async(time.sleep, (10,))
    print(result.get(timeout=1))        # raises multiprocessing.TimeoutError

TimeoutError: 